# Output Operations on DStreams

Output operations allow DStream’s data to be pushed out to external systems like a database or a file systems. Since the output operations actually allow the transformed data to be consumed by external systems, they trigger the actual execution of all the DStream transformations (similar to actions for RDDs). Currently, the following output operations are defined:

| Output Operation        | Meaning           |
|-------------:|:------------- |
| **pprint**()      | Prints the first ten elements of every batch of data in a DStream on the driver node running the streaming application. This is useful for development and debugging. |
| **saveAsTextFiles**(prefix, [suffix])     | Save this DStream's contents as text files. The file name at each batch interval is generated based on prefix and suffix: "prefix-TIME_IN_MS[.suffix]". |
| **foreachRDD**(func) | The most generic output operator that applies a function, func, to each RDD generated from the stream. This function should push the data in each RDD to an external system, such as saving the RDD to files, or writing it over the network to a database. Note that the function func is executed in the driver process running the streaming application, and will usually have RDD actions in it that will force the computation of the streaming RDDs. |



In the Java and Scala APIs for Spark, there are also the `saveAsObjectFiles` and `saveAsHadoopFiles`. Unfortunately, these are not available in the Python API


### Demo
The following is a demonstration of how to output RDDs.

In [2]:
"""
import findspark
# TODO: your path will likely not have 'matthew' in it. Change it to reflect your path.
findspark.init('/home/siddharth/spark-2.1.0-bin-hadoop2.7')
"""

"\nimport findspark\n# TODO: your path will likely not have 'matthew' in it. Change it to reflect your path.\nfindspark.init('/home/siddharth/spark-2.1.0-bin-hadoop2.7')\n"

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=8fb607022efd9807975ba65861c04a0cc89486da1198d14c2ab8cc372f2c3599
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [3]:
import sys
import pyspark
import pyspark.streaming
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [7]:
sc = SparkContext(master="local[2]", appName="WordcountToTextFiles")
ssc = StreamingContext(sc, 5)
    
lines = ssc.textFileStream('DrSeuss.text')

def linesplit(line):
    return line.split(" ")

counts = lines.flatMap(linesplit).map(lambda x: (x, 1)).reduceByKey(lambda a, b: a+b)
counts.saveAsTextFiles("output/Counts")

In [8]:
ssc.start()

In [9]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)#--files are generated and saved insid output/Counts

## References
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#output-operations-on-dstreams
